# Fine-tuning Embeddings for RAG on Specific Data

As we start our "fine-tuning" week, we'll start with the lowest hanging improvement one can do for RAG - which is:

Fine-tuning embeddings!

- 🤝 Breakout Room #1:
  - Task 1: Dependencies and Boilerplate
  - Task 2: Loading Data
  - Task 3: Constructing a Fine-tuning Dataset
  - Task 4: Fine-tuning `snowflake-arctic-embed-l`
  - Task 5: Evaluating our Retriever



#### Basic Overview of Fine-tuning Embeddings

In essence, what we want to do when we fine-tune our embedding models is very simple:

```
Move the embeddings for questions relating to a document
closer together with that document
```

We can think of fine-tuning our embedding models as follows:

1) We have some pair of text items that *should* be closer together
  - `Question`, `Document` pairs
  - EX: `Who drives the bus?`, `The bus was driven by Kyle, the Bus Driver`.

2) We use these pairs as labeled data to fine-tune our embedding model.

The process of training helps the model more accurately associate our questions with the correct documents.

#####❓ Question #1:

Describe the nuance between using Q&D pairs to train the embedding model vs. inter-document pairs/related sentences.

What caveats does this approach have? Are there any special considerations for what kind of Q's we should use?

---

**ANSWER:**

We are specifically relating *the questions* to *the documents*. This means that we are making our embedding model at the very specific task of relating potential questions to specific documents.

There are many caveats, but the main ones are:

- Your Q's should reflect the Q's of your users
- This kind of fine-tuning will (purposefully) "overfit" on your data; this is the desired result in this case.

## Task 1: Dependencies and Boilerplate

We'll set up our `nest_asyncio` so we can leverage async loops in our Notebook.

We'll also install the required libraries we'll be using today, and set up our OpenAI API key!

### Nest Asyncio

In [2]:
import nest_asyncio

nest_asyncio.apply()

### Install Dependencies

>> NOTE: You do not need to do these steps if you are running this notebook locally with `uv`.

In [3]:
#!pip install -qU langchain_openai langchain_huggingface langchain_core langchain langchain_community langchain-text-splitters

In [4]:
#!pip install -qU faiss-cpu python-pptx==1.0.2 nltk==3.9.1 pymupdf beautifulsoup4 lxml 

### Provide OpenAI API Key

In [5]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key: ")

## Task 2: Loading Data

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [6]:
!mkdir data

mkdir: data: File exists


In [7]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31440    0 31440    0     0   106k      0 --:--:-- --:--:-- --:--:--  106k


In [8]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70299    0 70299    0     0   196k      0 --:--:-- --:--:-- --:--:--  196k


In [9]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import BSHTMLLoader

path = "data/"
text_loader = DirectoryLoader(path, glob="*.html", loader_cls=BSHTMLLoader)

Next, we'll set up a classic naive chunking strategy as we only care that the documents get parsed into chunks that we can generate synthetic questions about.

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 750,
    chunk_overlap  = 20,
    length_function = len
)

Next we can load/split these documents as follows.

>> NOTE: You may need to run this cell twice to get it to work.

In [11]:
training_documents = text_splitter.split_documents(text_loader.load())

In [12]:
len(training_documents)

102

Next, we're going to associate each of our chunks with a unique identifier.

In [13]:
import uuid

id_set = set()

for document in training_documents:
  id = str(uuid.uuid4())
  while id in id_set:
    id = uuid.uuid4()
  id_set.add(id)
  document.metadata["id"] = id

Next, we'll simply use naive Python slicing to create a training, test, and validation set to prepare our data for the next step.

In [14]:
training_split_documents = training_documents[:len(training_documents) - 24]
val_split_documents = training_documents[len(training_documents) - 24:102-12]
test_split_documents = training_documents[102-12:]

## Task 3: Constructing a Fine-tuning Dataset

Using the nodes we created above, we can finally start constructing a fine-tuning dataset utilizing OpenAI's `gpt-4o-mini` (released [today](https://openai.com/index/gpt-4o-mini-advancing-cost-efficient-intelligence/)).

The basic idea here is straightforward enough:

1. We look at a document
2. We generate questions that could be answered by that node

This gives us a number of question/context pairs that we can use to fine-tune our Embeddings model.

In [15]:
from langchain_openai import ChatOpenAI

qa_chat_model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

We'll create a simple Question Generation prompt to query `gpt-4o-mini` to generate Questions for each retrieved context.

In [16]:
from langchain_core.prompts import ChatPromptTemplate

qa_prompt = """\
Given the following context, you must generate questions based on only the provided context.

You are to generate {n_questions} questions which should be provided in the following format:

1. QUESTION #1
2. QUESTION #2
...

Context:
{context}
"""

qa_prompt_template = ChatPromptTemplate.from_template(qa_prompt)

We'll create a simple chain to query the LLM!

In [17]:
question_generation_chain = qa_prompt_template | qa_chat_model

There's a lot going on in this function - let's take a deeper look:

1. First, we provide a list of documents and a number of questions
2. We, for each document in our list, generate `n_questions` of questions.
3. We then associate those questions and contexts via a `UUID`.

> NOTE: The reason we're doing this `UUID` association is for ease of use later in the notebook.

##### 🏗️ Activity #1:

We have:

- Lists of `Documents` with the `metadata` field `id`.

We need:

- An object with key `id`, which have values `str` questions.
- An object with key `question_id`, which have values `List(str)` which will be a list of associated `context_id`.

An Example:

question_object:
```python
{
'b4b95fb6-f827-4454-aa5b-20e62733f172': 'What types of accessible formats are available for persons with disabilities?',
'df58ee4f-714c-419e-8324-94e5870574e2': 'How do accessible formats benefit persons with disabilities?',
'505fce8b-0e56-48de-a251-61027e396918': 'What are some of the risks associated with the increasing capabilities of AI systems that generate synthetic content?',
'8ff0ab33-60dc-4fee-8958-91bfb686aca8': 'Why is it important for providers of AI systems to embed technical solutions for marking and detecting synthetic content?'
}
 ```

 context_object:
 ```python
{
'b4b95fb6-f827-4454-aa5b-20e62733f172': ['dd75bf94-75f3-4603-8e4b-5522f6925638'],
'df58ee4f-714c-419e-8324-94e5870574e2': ['dd75bf94-75f3-4603-8e4b-5522f6925638'],
'505fce8b-0e56-48de-a251-61027e396918': ['ffe3893f-688c-48e8-90bd-7a9feb953d90'],
'8ff0ab33-60dc-4fee-8958-91bfb686aca8': ['ffe3893f-688c-48e8-90bd-7a9feb953d90'],
}
 ```

 As you can see, a piece of context can be associated with more than 1 question.

 The task is to write the Python function(s) to accomplish this task.

 Your function signature is provided below, along with the desired return values.

 > NOTE: You can make any modifications that you desire - assuming that you have the correct input and outputs.

In [18]:
import tqdm

async def create_questions(documents, n_questions):
    """
    Create questions for each document and track their relationships.
    
    Args:
        documents: List of Document objects with metadata containing 'id'
        n_questions: Number of questions to generate per document
        
    Returns:
        questions: Dict mapping question_id to question text
        relevant_docs: Dict mapping question_id to list of relevant document ids
    """
    questions = {}
    relevant_docs = {}
    
    # Process each document with progress bar
    for document in tqdm.tqdm(documents, desc="Processing documents"):
        # Generate questions for this document
        result = await question_generation_chain.ainvoke({
            "context": document.page_content,
            "n_questions": n_questions
        })
        
        # Parse the numbered list of questions
        question_list = result.content.strip().split('\n')
        for question in question_list:
            # Skip empty lines and extract actual question
            if not question.strip():
                continue
            # Remove the number prefix (e.g. "1. ")
            question_text = question.split('. ', 1)[-1].strip()
            if not question_text:
                continue
                
            # Generate unique ID for this question
            question_id = str(uuid.uuid4())
            
            # Store the question and its relationship to the document
            questions[question_id] = question_text
            relevant_docs[question_id] = [document.metadata["id"]]
    
    # Return the dictionaries directly without await
    return questions, relevant_docs

### REMOVE `await` IF NOT USING ASYNC (HINT: Use `async`)

In [19]:
training_questions, training_relevant_contexts = await create_questions(training_split_documents, 2)

Processing documents: 100%|██████████| 78/78 [01:22<00:00,  1.06s/it]


We'll use the function to generate training, validation, and test data.

In [20]:
val_questions, val_relevant_contexts = await create_questions(val_split_documents, 2)

Processing documents: 100%|██████████| 12/12 [00:12<00:00,  1.01s/it]


In [21]:
test_questions, test_relevant_contexts = await create_questions(test_split_documents, 2)

Processing documents: 100%|██████████| 12/12 [00:13<00:00,  1.11s/it]


### Reformating and Saving Datasets

Now, we can save our datasets for later use!

In [22]:
import json

training_corpus = {train_item.metadata["id"] : train_item.page_content for train_item in training_split_documents}

train_dataset = {
    "questions" : training_questions,
    "relevant_contexts" : training_relevant_contexts,
    "corpus" : training_corpus
}

with open("training_dataset.jsonl", "w") as f:
  json.dump(train_dataset, f)

In [23]:
val_corpus = {val_item.metadata["id"] : val_item.page_content for val_item in val_split_documents}

val_dataset = {
    "questions" : val_questions,
    "relevant_contexts" : val_relevant_contexts,
    "corpus" : val_corpus
}

with open("val_dataset.jsonl", "w") as f:
  json.dump(val_dataset, f)

In [24]:
train_corpus = {test_item.metadata["id"] : test_item.page_content for test_item in test_split_documents}

test_dataset = {
    "questions" : test_questions,
    "relevant_contexts" : test_relevant_contexts,
    "corpus" : train_corpus
}

with open("test_dataset.jsonl", "w") as f:
  json.dump(test_dataset, f)

## Task 4: Fine-tuning `snowflake-arctic-embed-l`

Now that we have a dataset, let's grab a `sentence-transformers` Embeddings model!

We'll be using Snowflake's [`snowflake-arctic-embed-l`](https://huggingface.co/Snowflake/snowflake-arctic-embed-l) as a base embeddings model.

It is a well performing embeddings model by itself, but there's a lot of very specific domain terms and vocabulary in our courpus - so lets fine-tune it and see what that can do for us!

>> NOTE: Skip installing dependencies if you are running this notebook locally.

In [25]:
#!pip install -qU sentence_transformers datasets pyarrow

In [26]:
from sentence_transformers import SentenceTransformer

model_id = "Snowflake/snowflake-arctic-embed-l"
model = SentenceTransformer(model_id)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


We'll grab some necessary imports from `sentence_transformers` and `torch`.

> NOTE: PyTorch (`torch`) is a popular machine learning library - while we don't go very deep into PyTorch it's an incredibly powerful and interesting library! Please read more about it [here](https://pytorch.org/tutorials/beginner/basics/intro.html)!

In [27]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sentence_transformers import InputExample

We're using a toy batch size here to reflect the limited number of examples we have.

> NOTE: It is typical to use a much larger batch size (~64+), hardware permitting.

In [28]:
BATCH_SIZE = 10

Let's move our dataset into the expected format for training.

In [29]:
corpus = train_dataset['corpus']
queries = train_dataset['questions']
relevant_docs = train_dataset['relevant_contexts']

examples = []
for query_id, query in queries.items():
    doc_id = relevant_docs[query_id][0]
    text = corpus[doc_id]
    example = InputExample(texts=[query, text])
    examples.append(example)

Now we can create a `torch` `DataLoader`!

In [30]:
loader = DataLoader(
    examples, batch_size=BATCH_SIZE
)

Next up, we'll prepare our loss function!

Loss is an important part of training, fine-tuning, and more. If you want a deep dive on loss - you can check out our [event on loss!](https://www.youtube.com/watch?v=iB8FWR9aD5Q&t=8s).

The core loss we're using today is called `MultipleNegativesRankingLoss` - you can find more information [here](https://github.com/UKPLab/sentence-transformers/blob/master/sentence_transformers/losses/MultipleNegativesRankingLoss.py).

This is "wrapped" in `MatryoshkaLoss`, which you can read the implementation of [here](https://github.com/UKPLab/sentence-transformers/blob/master/sentence_transformers/losses/MatryoshkaLoss.py).

In [31]:
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss

matryoshka_dimensions = [768, 512, 256, 128, 64]
inner_train_loss = MultipleNegativesRankingLoss(model)
train_loss = MatryoshkaLoss(
    model, inner_train_loss, matryoshka_dims=matryoshka_dimensions
)

##### 🏗️ Activity #2:

Both of these losses sound "cool", but what are they - exactly - under the hood?

Why are these losses specifically doing? Please write a short summary of each loss.

> NOTE: This is a course focused on AI Engineering and the application of AI - looking for a hint? Try pasting the code (linked above) into ChatGPT/Claude to write the summary!

** Answer **: 

MultipleNegativesRankingLoss: This loss function is commonly used in contrastive learning for embedding models. It encourages the model to bring positive pairs (e.g., a question and its relevant document) closer in the embedding space while pushing apart negative pairs (e.g., a question and an irrelevant document).

MatryoshkaLoss: This is a specialized loss wrapper that allows integrating multiple loss functions or applying a loss at various layers or dimensions of the model. The term "Matryoshka" refers to the nested nature of Russian nesting dolls, indicating the hierarchical application of loss components.

Applying loss at multiple dimensions ensures that the model doesn't just learn to differentiate at a single embedding size but builds a more layered understanding. This can help in capturing various levels of semantic meaning, which is crucial for tasks requiring nuanced understanding like RAG.


Now we can set-up our evaluator.

> NOTE: Due to the formatting of our dataset - this is all we have to do!

In [32]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator

corpus = val_dataset['corpus']
queries = val_dataset['questions']
relevant_docs = val_dataset['relevant_contexts']

evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs)

We'll train this model for 5 epochs, though you could increase this number if we had a significant amount more data.

In [33]:
EPOCHS = 5

It's training time!

> NOTE: We're manually defining a warm-up period here - this is just to provide a smooth ramp into our training!

In [34]:
import wandb
wandb.init(mode="disabled")

In [35]:
warmup_steps = int(len(loader) * EPOCHS * 0.1)

model.fit(
    train_objectives=[(loader, train_loss)],
    epochs=EPOCHS,
    warmup_steps=warmup_steps,
    output_path='finetuned_arctic_ft',
    show_progress_bar=True,
    evaluator=evaluator,
    evaluation_steps=50
)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss,Cosine Accuracy@1,Cosine Accuracy@3,Cosine Accuracy@5,Cosine Accuracy@10,Cosine Precision@1,Cosine Precision@3,Cosine Precision@5,Cosine Precision@10,Cosine Recall@1,Cosine Recall@3,Cosine Recall@5,Cosine Recall@10,Cosine Ndcg@10,Cosine Mrr@10,Cosine Map@100
16,No log,No log,0.916667,1.000000,1.000000,1.000000,0.916667,0.333333,0.200000,0.100000,0.916667,1.000000,1.000000,1.000000,0.969244,0.958333,0.958333
32,No log,No log,0.875000,1.000000,1.000000,1.000000,0.875000,0.333333,0.200000,0.100000,0.875000,1.000000,1.000000,1.000000,0.953866,0.937500,0.937500
48,No log,No log,0.875000,1.000000,1.000000,1.000000,0.875000,0.333333,0.200000,0.100000,0.875000,1.000000,1.000000,1.000000,0.953866,0.937500,0.937500
50,No log,No log,0.875000,1.000000,1.000000,1.000000,0.875000,0.333333,0.200000,0.100000,0.875000,1.000000,1.000000,1.000000,0.953866,0.937500,0.937500
64,No log,No log,0.916667,1.000000,1.000000,1.000000,0.916667,0.333333,0.200000,0.100000,0.916667,1.000000,1.000000,1.000000,0.969244,0.958333,0.958333
80,No log,No log,0.916667,1.000000,1.000000,1.000000,0.916667,0.333333,0.200000,0.100000,0.916667,1.000000,1.000000,1.000000,0.969244,0.958333,0.958333


In [37]:
from huggingface_hub import notebook_login

notebook_login()

In [40]:
hf_username = "wkoleilat-happytitan"

In [41]:
model.push_to_hub(f"{hf_username}/legal-ft-2")

model.safetensors: 100%|██████████| 1.34G/1.34G [00:55<00:00, 23.9MB/s]


'https://huggingface.co/wkoleilat-happytitan/legal-ft-2/commit/3ef0d00665e156f22d9495d28c2ee96124230dcc'

## Task 5: Evaluating our Retriever

Now that we have fine-tuned our retriever - let's see if it's worthwhile!

We'll start with some basic imports.

In [42]:
import pandas as pd

from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.documents import Document

Now we'll define a function that will help us evaluate our retrieval process.

> NOTE: We're assuming 1 correct document in a "hit".

In [48]:
def evaluate_openai(
    dataset,
    embed_model,
    top_k=5,
    verbose=False,
):
  corpus = dataset['corpus']
  questions = dataset['questions']
  relevant_docs = dataset['relevant_contexts']
  documents = [Document(page_content=content, metadata={"id": doc_id}) for doc_id, content in corpus.items()]
  vectorstore = FAISS.from_documents(documents, embed_model)

  retriever = vectorstore.as_retriever(search_kwargs={"k": top_k})

  eval_results = []
  for id, question in tqdm.tqdm(questions.items()):
    retrieved_nodes = retriever.invoke(question)
    retrieved_ids = [node.metadata["id"] for node in retrieved_nodes]
    expected_id = relevant_docs[id][0]
    is_hit = expected_id in retrieved_ids
    eval_results.append({"id": id, "question": question, "expected_id": expected_id, "is_hit": is_hit})

  return eval_results

All that's left to do is evaluate, we'll evaluate our model against:

1. OpenAI's closed source `text-embedding-3-small`
2. The base non-fine-tuned version of `Snowflake/snowflake-arctic-embed-l`.

Let's see how it stacks up!

### `text-embedding-3-small`

In [49]:
te3_openai = OpenAIEmbeddings(model="text-embedding-3-small")
te3_results = evaluate_openai(test_dataset, te3_openai)

100%|██████████| 24/24 [00:09<00:00,  2.56it/s]


In [50]:
te3_results_df = pd.DataFrame(te3_results)

In [51]:
te3_hit_rate = te3_results_df["is_hit"].mean()
te3_hit_rate

np.float64(1.0)

### `Snowflake/snowflake-arctic-embed-l` (base)

In [53]:
from langchain_huggingface import HuggingFaceEmbeddings

huggingface_embeddings = HuggingFaceEmbeddings(model_name="Snowflake/snowflake-arctic-embed-l")
arctic_embed_m_results = evaluate_openai(test_dataset, huggingface_embeddings)

100%|██████████| 24/24 [00:37<00:00,  1.58s/it]


In [54]:
arctic_embed_m_results_df = pd.DataFrame(arctic_embed_m_results)

In [55]:
arctic_embed_m_hit_rate = arctic_embed_m_results_df["is_hit"].mean()
arctic_embed_m_hit_rate

np.float64(0.8333333333333334)

### `Snowflake/snowflake-arctic-embed-l` (fine-tuned)

In [56]:
finetune_embeddings = HuggingFaceEmbeddings(model_name="finetuned_arctic_ft")
finetune_results = evaluate_openai(test_dataset, finetune_embeddings)

Some weights of BertModel were not initialized from the model checkpoint at finetuned_arctic_ft and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 24/24 [18:15<00:00, 45.63s/it]


In [57]:
finetune_results_df = pd.DataFrame(finetune_results)

In [58]:
finetune_hit_rate = finetune_results_df["is_hit"].mean()
finetune_hit_rate

np.float64(1.0)

## Task 1: Vibe Checking the RAG Pipeline

We're going to use our RAG pipeline to vibe check on some common phrases now that we've modified it!

### Creating New Chunks

In order to try and evaluate our system more fairly, let's create new chunks that we will use to create our Vector Store.

In [59]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap  = 50,
    length_function = len
)

training_documents = text_splitter.split_documents(text_loader.load())

### Base Chain

We'll start by constructing our base chain, which will use the untrained retrieval model.

#### R - Retrieval

In [60]:
from langchain_community.vectorstores import FAISS

base_vectorstore = FAISS.from_documents(training_documents, huggingface_embeddings)
base_retriever = base_vectorstore.as_retriever(search_kwargs={"k": 6})

#### A - Augmented

In [61]:
from langchain_core.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and a question, you must answer the question. If you do not know the answer, you must state that you do not know.

Context:
{context}

Question:
{question}

Answer:
"""

rag_prompt_template = ChatPromptTemplate.from_template(RAG_PROMPT)

#### G - Generation

In [62]:
rag_llm =  ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

#### RAG - LCEL RAG Pipeline

In [63]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

base_rag_chain = (
    {"context": itemgetter("question") | base_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt_template | rag_llm | StrOutputParser(), "context": itemgetter("context")}
)

In [64]:
base_rag_chain.invoke({"question" : "What is an agent?"})["response"]

'An agent, in the context of AI, is an infuriatingly vague term that generally refers to AI systems that can act on your behalf. There are different interpretations of what an agent is, with some viewing them as systems that autonomously perform tasks (like a travel agent), while others see them as LLMs (large language models) that utilize tools to solve problems. However, the term lacks a clear and widely understood definition, leading to confusion about its meaning and utility.'

In [65]:
base_rag_chain.invoke({"question" : "Who has produced better models than GPT-3?"})["response"]

'Organizations that have produced better-than-GPT-3 class models include Anthropic, Mistral, Google, Meta, EleutherAI, Stability AI, TII in Abu Dhabi (Falcon), Microsoft Research, xAI, Replit, Baidu, and several others.'

In [66]:
base_rag_chain.invoke({"question" : "What is the laziest AI month?"})["response"]

'I do not know.'

In [67]:
base_rag_chain.invoke({"question" : "What is the largest model that Simon has run on his phone?"})["response"]

'I do not know.'

### Fine-tuned Embedding Model

Now let's rebuild our RAG chain with the Fine-tuned model - the only component we need to change is our `FAISS` vectorstore!

In [68]:
finetune_vectorstore = FAISS.from_documents(training_documents, finetune_embeddings)
finetune_retriever = finetune_vectorstore.as_retriever(search_kwargs={"k": 6})

In [69]:
finetune_rag_chain = (
    {"context": itemgetter("question") | finetune_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt_template | rag_llm | StrOutputParser(), "context": itemgetter("context")}
)

In [70]:
finetune_rag_chain.invoke({"question" : "What is an Agent?"})["response"]

'An "agent" is a term that lacks a single, clear, and widely understood meaning in the context of AI. It is often used to refer to AI systems that can act on behalf of a user, but there are various interpretations of what this entails. Some people view agents as systems that can autonomously perform tasks, while others think of them as LLMs (large language models) that utilize tools to solve problems. The term is often associated with concepts like autonomy, but there is significant ambiguity surrounding its definition. Overall, the concept of agents remains frustratingly vague and has not yet seen widespread practical implementation.'

In [71]:
finetune_rag_chain.invoke({"question" : "Who has produced better models than GPT-3?"})["response"]

'Organizations that have produced better-than-GPT-3 class models include Anthropic, Mistral, Google, Meta, EleutherAI, Stability AI, TII in Abu Dhabi (Falcon), Microsoft Research, xAI, Replit, Baidu, and several others.'

In [72]:
finetune_rag_chain.invoke({"question" : "What is the laziest AI month?"})["response"]

'I do not know.'

In [73]:
finetune_rag_chain.invoke({"question" : "What is the largest model that Simon has run on his phone?"})["response"]

'The largest model that Simon has run on his phone is the Llama 2 model, specifically the Mistral 7B model.'

####❓Question #2:

Which LCEL RAG Chain do you think answered the questions better, and why?


The fine tuned model performed a little better. It answered a question that the base model didn't know the answer to.

But also the answer to the first question was more accurate and more relevant.

Though need to have a more regorous testing of the model to assess the performance objectively.

## Task 2: RAGAS Evaluation

It's great to have some idea of how our system is doing based on vibe-checks, but let's use RAGAS to provide more insight info. on how things are improving!

> NOTE: Please recreate *exactly* the RAGAS process we used to evaluate RAG, baselining with the default retriever, and then comparing the new retriever. The includes the Synthetic Data Generation steps.

In [77]:
### YOUR CODE HERE
import getpass

os.environ["RAGAS_APP_TOKEN"] = getpass.getpass("Please enter your Ragas API key!")

In [78]:
!mkdir data

Python(47742) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [79]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

Python(47743) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [80]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

Python(47755) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [81]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

Python(47766) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


In [82]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings()) 

In [83]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Generating Samples: 100%|██████████| 12/12 [01:07<00:00,  5.61s/it]


In [84]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,Wht is Googel's role in LLMs?,[Code may be the best application The ethics o...,Google is one of the organizations that have p...,single_hop_specifc_query_synthesizer
1,How does the concept of 'vibes-based developme...,[Based Development As a computer scientist and...,The concept of 'vibes-based development' refle...,single_hop_specifc_query_synthesizer
2,what gpt-4 do?,[Simon Willison’s Weblog Subscribe Stuff we fi...,We don’t yet know how to build GPT-4.,single_hop_specifc_query_synthesizer
3,What are some insights on DALL-E 3 from the bl...,[easy to follow. The rest of the document incl...,One of the blog posts in 2023 discusses 'Promp...,single_hop_specifc_query_synthesizer
4,What are the ethical concerns related to the t...,[<1-hop>\n\nCode may be the best application T...,The ethical concerns related to the training d...,multi_hop_abstract_query_synthesizer
5,How have advancements in model training costs ...,[<1-hop>\n\nCode may be the best application T...,Advancements in model training costs and cost ...,multi_hop_abstract_query_synthesizer
6,Why are Large Language Models (LLMs) considere...,[<1-hop>\n\nCode may be the best application T...,Large Language Models (LLMs) are considered bl...,multi_hop_abstract_query_synthesizer
7,How do the challenges of understanding and con...,[<1-hop>\n\nCode may be the best application T...,The challenges of understanding and controllin...,multi_hop_abstract_query_synthesizer
8,How do the advancements in GPT-4 and GPT-4o re...,[<1-hop>\n\nit easier for a model to follow th...,The advancements in GPT-4 and GPT-4o highlight...,multi_hop_specific_query_synthesizer
9,How did the Claude 3 series from Anthropic and...,[<1-hop>\n\nVoice and live camera mode are sci...,"The Claude 3 series from Anthropic, launched i...",multi_hop_specific_query_synthesizer


In [85]:
dataset.upload()

'https://app.ragas.io/dashboard/alignment/testset/8311e407-87df-4e5f-9a1d-c4cc800b3c18'

In [86]:
path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


In [87]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_documents = text_splitter.split_documents(docs)
len(split_documents)

74

Running the lCEL RAG Pipeline with the base embeddings model

In [88]:
from langchain_openai import OpenAIEmbeddings

base_embeddings = HuggingFaceEmbeddings(model_name="Snowflake/snowflake-arctic-embed-l")

## base_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [90]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="ai_across_years",
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="ai_across_years",
    embedding=base_embeddings,
)

In [91]:
_ = vector_store.add_documents(documents=split_documents)

In [92]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

In [93]:
def retrieve(state):
  retrieved_docs = retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

In [94]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
You are a helpful assistant who answers questions based on provided context. You must only use the provided context, and cannot use your own knowledge.

### Question
{question}

### Context
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

In [95]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [96]:
def generate(state):
  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = rag_prompt.format_messages(question=state["question"], context=docs_content)
  response = llm.invoke(messages)
  return {"response" : response.content}

In [97]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

class State(TypedDict):
  question: str
  context: List[Document]
  response: str

In [98]:
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [99]:
response = graph.invoke({"question" : "How are LLM agents useful?"})

In [100]:
response["response"]

"LLM agents can be useful despite their inherent unreliability and the challenges associated with them. While many people harbor inaccurate mental models about these tools and some may dismiss their utility entirely due to their flaws, there are genuine applications for LLMs. The key skill in leveraging LLMs effectively lies in understanding how to navigate their limitations while utilizing their potential power.\n\nTo harness the value of LLMs, it's essential to educate users about their good applications and provide guidance on how to apply these tools successfully. This involves acknowledging the complexities and intuitively difficult aspects of working with LLMs to avoid common pitfalls. By doing so, decision-makers can make informed choices about implementing LLMs in various contexts, ultimately leading to beneficial outcomes. Therefore, LLM agents have the potential to offer significant value when approached with a thoughtful and informed strategy."

In [101]:
for test_row in dataset:
  response = graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [102]:
dataset.to_pandas()

,user_input,retrieved_contexts,reference_contexts,response,reference,synthesizer_name
0,Wht is Googel's role in LLMs?,[If we want people with decision-making author...,[Code may be the best application The ethics o...,The provided context does not explicitly menti...,Google is one of the organizations that have p...,single_hop_specifc_query_synthesizer
1,How does the concept of 'vibes-based developme...,"[Meanwhile, it’s increasingly common for end u...",[Based Development As a computer scientist and...,The concept of 'vibes-based development' refle...,The concept of 'vibes-based development' refle...,single_hop_specifc_query_synthesizer
2,what gpt-4 do?,[Article Visitors Pageviews Bing: “I will not ...,[Simon Willison’s Weblog Subscribe Stuff we fi...,The provided context does not explicitly detai...,We don’t yet know how to build GPT-4.,single_hop_specifc_query_synthesizer
3,What are some insights on DALL-E 3 from the bl...,[Article Visitors Pageviews Bing: “I will not ...,[easy to follow. The rest of the document incl...,The provided context does not specifically men...,One of the blog posts in 2023 discusses 'Promp...,single_hop_specifc_query_synthesizer
4,What are the ethical concerns related to the t...,[Article Visitors Pageviews Bing: “I will not ...,[<1-hop>\n\nCode may be the best application T...,The ethical concerns related to the training d...,The ethical concerns related to the training d...,multi_hop_abstract_query_synthesizer
5,How have advancements in model training costs ...,[Article Visitors Pageviews Bing: “I will not ...,[<1-hop>\n\nCode may be the best application T...,Advancements in model training costs and the r...,Advancements in model training costs and cost ...,multi_hop_abstract_query_synthesizer
6,Why are Large Language Models (LLMs) considere...,[Article Visitors Pageviews Bing: “I will not ...,[<1-hop>\n\nCode may be the best application T...,Large Language Models (LLMs) are considered bl...,Large Language Models (LLMs) are considered bl...,multi_hop_abstract_query_synthesizer
7,How do the challenges of understanding and con...,[Article Visitors Pageviews Bing: “I will not ...,[<1-hop>\n\nCode may be the best application T...,The challenges of understanding and controllin...,The challenges of understanding and controllin...,multi_hop_abstract_query_synthesizer
8,How do the advancements in GPT-4 and GPT-4o re...,[Simon Willison’s Weblog\n\nSubscribe\n\nThing...,[<1-hop>\n\nit easier for a model to follow th...,The advancements in GPT-4 and GPT-4o as discus...,The advancements in GPT-4 and GPT-4o highlight...,multi_hop_specific_query_synthesizer
9,How did the Claude 3 series from Anthropic and...,[Posted 31st December 2023 at 11:59 pm · Follo...,[<1-hop>\n\nVoice and live camera mode are sci...,The Claude 3 series from Anthropic and advance...,"The Claude 3 series from Anthropic, launched i...",multi_hop_specific_query_synthesizer


In [103]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())

In [104]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))

In [105]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=360)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating: 100%|██████████| 72/72 [12:29<00:00, 10.41s/it]


{'context_recall': 0.3500, 'faithfulness': 0.9231, 'factual_correctness': 0.4058, 'answer_relevancy': 0.7306, 'context_entity_recall': 0.2209, 'noise_sensitivity_relevant': 0.2028}

Running the LCEL RAG Pipeline with the fine-tuned embeddings model.

In [106]:
fine_tuned_embeddings = HuggingFaceEmbeddings(model_name="finetuned_arctic_ft")

Some weights of BertModel were not initialized from the model checkpoint at finetuned_arctic_ft and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [107]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="ai_across_years_fine_tuned",
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="ai_across_years_fine_tuned",
    embedding=fine_tuned_embeddings,
)

In [108]:
_ = vector_store.add_documents(documents=split_documents)

In [109]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

In [110]:
def retrieve(state):
  retrieved_docs = retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

In [111]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
You are a helpful assistant who answers questions based on provided context. You must only use the provided context, and cannot use your own knowledge.

### Question
{question}

### Context
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

In [112]:


llm = ChatOpenAI(model="gpt-4o-mini")

In [113]:
def generate(state):
  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = rag_prompt.format_messages(question=state["question"], context=docs_content)
  response = llm.invoke(messages)
  return {"response" : response.content}

In [114]:
class State(TypedDict):
  question: str
  context: List[Document]
  response: str

In [115]:
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [116]:
response = graph.invoke({"question" : "How are LLM agents useful?"})

In [117]:
response["response"]

'LLM agents can be useful primarily in the domain of writing code. They have shown remarkable capability in generating surprisingly competent code, leveraging the relatively simpler grammar rules of programming languages compared to natural languages. Additionally, LLMs are versatile tools that can answer questions, summarize documents, translate languages, and extract information, thereby boosting productivity and quality of life for users who learn to utilize them effectively. However, there is a need for guidance to navigate the best practices and avoid the potential pitfalls associated with their use.'

In [118]:
for test_row in dataset:
  response = graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [119]:
dataset.to_pandas()

,user_input,retrieved_contexts,reference_contexts,response,reference,synthesizer_name
0,Wht is Googel's role in LLMs?,[If we want people with decision-making author...,[Code may be the best application The ethics o...,The provided context does not explicitly menti...,Google is one of the organizations that have p...,single_hop_specifc_query_synthesizer
1,How does the concept of 'vibes-based developme...,"[Still, I’m surprised that no-one has beaten t...",[Based Development As a computer scientist and...,The concept of 'vibes-based development' highl...,The concept of 'vibes-based development' refle...,single_hop_specifc_query_synthesizer
2,what gpt-4 do?,[This is a huge advantage for open over closed...,[Simon Willison’s Weblog Subscribe Stuff we fi...,GPT-4 is an advanced language model released b...,We don’t yet know how to build GPT-4.,single_hop_specifc_query_synthesizer
3,What are some insights on DALL-E 3 from the bl...,[Posted 31st December 2023 at 11:59 pm · Follo...,[easy to follow. The rest of the document incl...,The blog posts from 2023 highlight several ins...,One of the blog posts in 2023 discusses 'Promp...,single_hop_specifc_query_synthesizer
4,What are the ethical concerns related to the t...,[Law is not ethics. Is it OK to train models o...,[<1-hop>\n\nCode may be the best application T...,The ethical concerns related to the training d...,The ethical concerns related to the training d...,multi_hop_abstract_query_synthesizer
5,How have advancements in model training costs ...,[Simon Willison’s Weblog\n\nSubscribe\n\nThing...,[<1-hop>\n\nCode may be the best application T...,Advancements in model training costs and cost ...,Advancements in model training costs and cost ...,multi_hop_abstract_query_synthesizer
6,Why are Large Language Models (LLMs) considere...,"[Still, I’m surprised that no-one has beaten t...",[<1-hop>\n\nCode may be the best application T...,Large Language Models (LLMs) are considered bl...,Large Language Models (LLMs) are considered bl...,multi_hop_abstract_query_synthesizer
7,How do the challenges of understanding and con...,[Code may be the best application\n\nThe ethic...,[<1-hop>\n\nCode may be the best application T...,The challenges of understanding and controllin...,The challenges of understanding and controllin...,multi_hop_abstract_query_synthesizer
8,How do the advancements in GPT-4 and GPT-4o re...,[Simon Willison’s Weblog\n\nSubscribe\n\nThing...,[<1-hop>\n\nit easier for a model to follow th...,"The advancements in GPT-4 and GPT-4o, as highl...",The advancements in GPT-4 and GPT-4o highlight...,multi_hop_specific_query_synthesizer
9,How did the Claude 3 series from Anthropic and...,[Getting back to models that beat GPT-4: Anthr...,[<1-hop>\n\nVoice and live camera mode are sci...,The Claude 3 series from Anthropic significant...,"The Claude 3 series from Anthropic, launched i...",multi_hop_specific_query_synthesizer


In [120]:
evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())

In [121]:
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))

In [122]:
custom_run_config = RunConfig(timeout=360)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating: 100%|██████████| 72/72 [13:32<00:00, 11.28s/it]


{'context_recall': 0.5667, 'faithfulness': 0.8600, 'factual_correctness': 0.3530, 'answer_relevancy': 0.8791, 'context_entity_recall': 0.5255, 'noise_sensitivity_relevant': 0.1515}

** Results **
Fine-Tuned Model shows higher Context Recall, Answer Relevancy, and Context Entity Recall, indicating better performance in retrieving relevant contexts and producing relevant answers.


Base Model excels in Faithfulness and Factual Correctness, suggesting it maintains more accurate and reliable information based on the provided contexts.


Noise Sensitivity Relevant is slightly lower for the Fine-Tuned Model, which may imply it is less sensitive to irrelevant or noisy information compared to the Base Model.

| Metric | Fine-Tuned Model | Base Model |
|------------------------------|----------------------|-----------------|
| Context Recall | 0.5667 | 0.3500 |
| Faithfulness | 0.8600 | 0.9231 |
| Factual Correctness | 0.3530 | 0.4058 |
| Answer Relevancy | 0.8791 | 0.7306 |
| Context Entity Recall | 0.5255 | 0.2209 |
| Noise Sensitivity Relevant | 0.1515 | 0.2028 |